## 1. Initial Setup and Data Cleaning
Imports, cleaning up the dataset, filtering for features, discritizing the data, test-train splitting.

In [1]:
import numpy as np
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import HillClimbSearch, ExpectationMaximization
from sklearn.model_selection import train_test_split
from pgmpy.inference import VariableElimination
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

c:\Users\nghis\miniforge3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dfDict = {
    'age': [], 'bp': [], 'sg': [], 'al': [], 'su': [],
    'rbc': [], 'pc': [], 'pcc': [], 'ba': [], 'bgr': [],
    'bu': [], 'sc': [], 'sod': [], 'pot': [], 'hemo': [],
    'pcv': [], 'wbcc': [], 'rbcc': [], 'htn': [], 'dm': [],
    'cad': [], 'appet': [], 'pe': [], 'ane': [], 'ckd': []
}
mapIndexToKey = dict(zip(np.arange(25), dfDict.keys()))
with open("dataset/chronic_kidney_disease.arff", "r") as f:
    for line in f:
        if line[0] == '@':
            continue
        line = line.strip()
        if line == '':
            continue
        line = line.replace('\t', '').split(',')
        index = 0
        for item in line:
            if item == '':
                continue
            if item == '?':
                dfDict[mapIndexToKey[index]].append(np.nan)
            elif index == 21:
                dfDict[mapIndexToKey[index]].append(int(item == 'good'))
            elif index == 24:
                dfDict[mapIndexToKey[index]].append(int(item == 'ckd'))
            elif index in [5, 6]:
                dfDict[mapIndexToKey[index]].append(int(item == 'normal'))
            elif index in [7, 8]:
                dfDict[mapIndexToKey[index]].append(int(item == 'present'))
            elif index in [18, 19, 20, 22, 23]:
                dfDict[mapIndexToKey[index]].append(int(item == 'yes'))
            else:
                dfDict[mapIndexToKey[index]].append(float(item))
            index += 1
df = pd.DataFrame(dfDict)
df

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,ckd
0,48.0,80.0,1.020,1.0,0.0,NaN,1.0,0.0,0.0,121.0,...,44.0,7800.0,5.2,1.0,1.0,0.0,1.0,0.0,0.0,1
1,7.0,50.0,1.020,4.0,0.0,NaN,1.0,0.0,0.0,NaN,...,38.0,6000.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1
2,62.0,80.0,1.010,2.0,3.0,1.0,1.0,0.0,0.0,423.0,...,31.0,7500.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,1
3,48.0,70.0,1.005,4.0,0.0,1.0,0.0,1.0,0.0,117.0,...,32.0,6700.0,3.9,1.0,0.0,0.0,0.0,1.0,1.0,1
4,51.0,80.0,1.010,2.0,0.0,1.0,1.0,0.0,0.0,106.0,...,35.0,7300.0,4.6,0.0,0.0,0.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,1.0,1.0,0.0,0.0,140.0,...,47.0,6700.0,4.9,0.0,0.0,0.0,1.0,0.0,0.0,0
396,42.0,70.0,1.025,0.0,0.0,1.0,1.0,0.0,0.0,75.0,...,54.0,7800.0,6.2,0.0,0.0,0.0,1.0,0.0,0.0,0
397,12.0,80.0,1.020,0.0,0.0,1.0,1.0,0.0,0.0,100.0,...,49.0,6600.0,5.4,0.0,0.0,0.0,1.0,0.0,0.0,0
398,17.0,60.0,1.025,0.0,0.0,1.0,1.0,0.0,0.0,114.0,...,51.0,7200.0,5.9,0.0,0.0,0.0,1.0,0.0,0.0,0


In [3]:
cleanedDf = df[['age', 'bp', 'su', 'rbc', 'bgr', 'sod', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'ckd']]
for col in cleanedDf.columns:
    print(f'{col}: {df[col].unique()}')

age: [48.  7. 62. 51. 60. 68. 24. 52. 53. 50. 63. 40. 47. 61. 21. 42. 75. 69.
 nan 73. 70. 65. 76. 72. 82. 46. 45. 35. 54. 11. 59. 67. 15. 55. 44. 26.
 64. 56.  5. 74. 38. 58. 71. 34. 17. 12. 43. 41. 57.  8. 39. 66. 81. 14.
 27. 83. 30.  4.  3.  6. 32. 80. 49. 90. 78. 19.  2. 33. 36. 37. 23. 25.
 20. 29. 28. 22. 79.]
bp: [ 80.  50.  70.  90.  nan 100.  60. 110. 140. 180. 120.]
su: [ 0.  3.  4.  1. nan  2.  5.]
rbc: [nan  1.  0.]
bgr: [121.  nan 423. 117. 106.  74. 100. 410. 138.  70. 490. 380. 208.  98.
 157.  76.  99. 114. 263. 173.  95. 108. 156. 264. 123.  93. 107. 159.
 140. 171. 270.  92. 137. 204.  79. 207. 124. 144.  91. 162. 246. 253.
 141. 182.  86. 150. 146. 425. 112. 250. 360. 163. 129. 133. 102. 158.
 165. 132. 104. 127. 415. 169. 251. 109. 280. 210. 219. 295.  94. 172.
 101. 298. 153.  88. 226. 143. 115.  89. 297. 233. 294. 323. 125.  90.
 308. 118. 224. 128. 122. 214. 213. 268. 256.  84. 105. 288. 139.  78.
 273. 242. 424. 303. 148. 160. 192. 307. 220. 447. 309.  22. 111.

In [4]:
cleanedDf['ckd'].value_counts()

ckd
1    250
0    150
Name: count, dtype: int64

In [5]:
# Discritizing the continuous variables
cleanedDf['age'] = cleanedDf['age'].apply(lambda age: 'Young' if age < 40 else 'Middle-Age' if age < 60 else 'Senior')
cleanedDf['bp'] = cleanedDf['bp'].apply(lambda bp: 'Normal' if bp < 80 else 'Stage1' if bp < 90 else 'Stage2')
cleanedDf['bgr'] = cleanedDf['bgr'].apply(lambda bgr: 'Normal' if bgr < 100 else 'Prediabetic' if bgr < 126 else 'Diabetic')
cleanedDf['sod'] = cleanedDf['sod'].apply(lambda sod: 'Low' if sod < 135 else 'Normal' if sod < 146 else 'High')

C:\Users\nghis\AppData\Local\Temp\ipykernel_26472\973289822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanedDf['age'] = cleanedDf['age'].apply(lambda age: 'Young' if age < 40 else 'Middle-Age' if age < 60 else 'Senior')
C:\Users\nghis\AppData\Local\Temp\ipykernel_26472\973289822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanedDf['bp'] = cleanedDf['bp'].apply(lambda bp: 'Normal' if bp < 80 else 'Stage1' if bp < 90 else 'Stage2')
C:\Users\nghis\AppData\Local\Temp\ipykernel_26472\97328982

In [6]:
X = cleanedDf.drop(['ckd'], axis = 1)
y = cleanedDf['ckd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 150)
X_train['ckd'] = y_train
# Create missing values in the test set for 'ckd'
X_test['ckd'] = [np.nan] * len(X_test)

## 2. Training the initial model
Use hill climb search to find the best DAG based on BIC-D scoring method, train the model as a discrete bayesian network using expectation maximization. Use variable elimination and probabilistic imputation to fill in missing values in the test set split and create our y_pred.

In [7]:
def get_EM(trainData, scoring = 'bic-d'):
    """
    Trains a Discrete Bayesian Network using Hill Climb Search to find the best DAG structure
    and Expectation Maximization to fit the model parameters.
    """
    hc = HillClimbSearch(trainData)
    dag = hc.estimate(scoring_method = scoring)

    model = DiscreteBayesianNetwork(dag.edges())

    em = ExpectationMaximization(model, trainData)
    em.model.fit(trainData)
    return em

In [8]:
em = get_EM(X_train)
print(em.model.get_cpds('ckd'))

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'C', 'bp': 'C', 'su': 'N', 'rbc': 'N', 'bgr': 'C', 'sod': 'C', 'htn': 'N', 'dm': 'N', 'cad': 'N', 'appet': 'N', 'pe': 'N', 'ane': 'N', 'ckd': 'N'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'C', 'bp': 'C', 'su': 'N', 'rbc': 'N', 'bgr': 'C', 'sod': 'C', 'htn': 'N', 'dm': 'N', 'cad': 'N', 'appet': 'N', 'pe': 'N', 'ane': 'N', 'ckd': 'N'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'C', 'bp': 'C', 'su': 'N', 'rbc': 'N', 'bgr': 'C', 'sod': 'C', 'htn': 'N', 'dm': 'N', 'cad': 'N', 'appet': 'N', 'pe': 'N', 'ane': 'N', 'ckd': 'N'}
  0%|          | 22/1000000 [00:03<43:22:01,  6.41it/s]
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'C', 'bp': 'C', 'su': 'N', 'rbc': 'N', 'bgr': 'C', 

+--------+----------+---------------------+----------+----------+
| htn    | htn(0.0) | htn(0.0)            | htn(1.0) | htn(1.0) |
+--------+----------+---------------------+----------+----------+
| rbc    | rbc(0.0) | rbc(1.0)            | rbc(0.0) | rbc(1.0) |
+--------+----------+---------------------+----------+----------+
| ckd(0) | 0.0      | 0.8990825688073395  | 0.0      | 0.0      |
+--------+----------+---------------------+----------+----------+
| ckd(1) | 1.0      | 0.10091743119266056 | 1.0      | 1.0      |
+--------+----------+---------------------+----------+----------+


In [9]:
em.model.edges

OutEdgeView([('rbc', 'bgr'), ('rbc', 'bp'), ('rbc', 'sod'), ('rbc', 'age'), ('rbc', 'ckd'), ('rbc', 'dm'), ('rbc', 'htn'), ('rbc', 'appet'), ('rbc', 'su'), ('rbc', 'pe'), ('rbc', 'ane'), ('rbc', 'cad'), ('ckd', 'dm'), ('ckd', 'appet'), ('ckd', 'pe'), ('ckd', 'ane'), ('ckd', 'sod'), ('ckd', 'bp'), ('dm', 'cad'), ('dm', 'bgr'), ('htn', 'ckd'), ('htn', 'age')])

In [10]:
# Impute missing values in X_test using Variable Elimination
def impute(em, X_test):
    infer = VariableElimination(em.model)
    X_test_new = X_test.copy()
    for idx, row in X_test_new.iterrows():
        evidence = {col: row[col] for col in row.index if pd.notna(row[col])}
        missing_vars = row[row.isna()].index.tolist()
        if missing_vars:
            imputed = infer.map_query(variables=missing_vars, evidence=evidence)
            for var in missing_vars:
                X_test_new.at[idx, var] = imputed[var]
    return X_test_new, infer

imputedXTest, infer = impute(em, X_test)
imputedXTest

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0i

,age,bp,su,rbc,bgr,sod,htn,dm,cad,appet,pe,ane,ckd
144,Senior,Stage2,0.0,0.0,Prediabetic,Normal,0.0,0.0,0.0,1.0,0.0,0.0,1.0
280,Middle-Age,Stage1,0.0,1.0,Normal,Normal,0.0,0.0,0.0,1.0,0.0,0.0,0.0
399,Middle-Age,Stage1,0.0,1.0,Diabetic,Normal,0.0,0.0,0.0,1.0,0.0,0.0,0.0
362,Young,Stage1,0.0,1.0,Normal,Normal,0.0,0.0,0.0,1.0,0.0,0.0,0.0
198,Middle-Age,Stage2,2.0,1.0,Diabetic,Normal,1.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,Senior,Normal,0.0,1.0,Normal,Low,1.0,0.0,0.0,1.0,0.0,0.0,1.0
269,Young,Stage1,0.0,1.0,Prediabetic,Normal,0.0,0.0,0.0,1.0,0.0,0.0,0.0
287,Young,Normal,0.0,1.0,Prediabetic,Normal,0.0,0.0,0.0,1.0,0.0,0.0,0.0
317,Middle-Age,Normal,0.0,1.0,Prediabetic,Normal,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## 3. Measure accuracy of initial model
Print out F1 scores and precision scores of our initial model

In [11]:
def scoringMethod(y_true, y_pred):
    f1_per_class = f1_score(y_true, y_pred, average=None)
    f1_micro = f1_score(y_true, y_pred, average='micro')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred)

    print("F1 score per class:", f1_per_class)
    print("Micro-average F1 score:", f1_micro)
    print("Macro-average F1 score:", f1_macro)
    print("Weighted-average F1 score:", f1_weighted)
    print("Precision:", precision)

scoringMethod(y_test, imputedXTest['ckd'])

F1 score per class: [0.88421053 0.92413793]
Micro-average F1 score: 0.9083333333333333
Macro-average F1 score: 0.9041742286751361
Weighted-average F1 score: 0.91016333938294
Precision: 1.0


## 4. Testing other hyperparameters (e.g scoring method)
Try discrete scoring method like bdeu, k2, bds, and aic-d to compare how our method did 

In [12]:
imputedXTestDict = {'bic-d': imputedXTest}
for method in ['k2', 'bdeu', 'bds', 'aic-d']:
    emTemp = get_EM(X_train, scoring = method)
    imputedXTestDict[method], _ = impute(emTemp, X_test)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'C', 'bp': 'C', 'su': 'N', 'rbc': 'N', 'bgr': 'C', 'sod': 'C', 'htn': 'N', 'dm': 'N', 'cad': 'N', 'appet': 'N', 'pe': 'N', 'ane': 'N', 'ckd': 'N'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'C', 'bp': 'C', 'su': 'N', 'rbc': 'N', 'bgr': 'C', 'sod': 'C', 'htn': 'N', 'dm': 'N', 'cad': 'N', 'appet': 'N', 'pe': 'N', 'ane': 'N', 'ckd': 'N'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'C', 'bp': 'C', 'su': 'N', 'rbc': 'N', 'bgr': 'C', 'sod': 'C', 'htn': 'N', 'dm': 'N', 'cad': 'N', 'appet': 'N', 'pe': 'N', 'ane': 'N', 'ckd': 'N'}
  0%|          | 75/1000000 [00:09<33:40:58,  8.25it/s]
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'C', 'bp': 'C', 'su': 'N', 'rbc': 'N', 'bgr': 'C', 

In [13]:
for method, imputedY in imputedXTestDict.items():
    print(method)
    scoringMethod(y_test, imputedY['ckd'])
    print()

bic-d
F1 score per class: [0.88421053 0.92413793]
Micro-average F1 score: 0.9083333333333333
Macro-average F1 score: 0.9041742286751361
Weighted-average F1 score: 0.91016333938294
Precision: 1.0

k2
F1 score per class: [0.56375839 0.28571429]
Micro-average F1 score: 0.4583333333333333
Macro-average F1 score: 0.4247363374880153
Weighted-average F1 score: 0.38302972195589646
Precision: 1.0

bdeu
F1 score per class: [0.86597938 0.90909091]
Micro-average F1 score: 0.8916666666666667
Macro-average F1 score: 0.8875351452671041
Weighted-average F1 score: 0.8940018744142455
Precision: 1.0

bds
F1 score per class: [0.80769231 0.85294118]
Micro-average F1 score: 0.8333333333333334
Macro-average F1 score: 0.8303167420814479
Weighted-average F1 score: 0.8371040723981902
Precision: 1.0

aic-d
F1 score per class: [0.89361702 0.93150685]
Micro-average F1 score: 0.9166666666666666
Macro-average F1 score: 0.9125619352958321
Weighted-average F1 score: 0.9182454095016029
Precision: 1.0



## 5. Answering our research question
Answer the quantitative questions we wrote at the start of the project

In [14]:
evidence = {
    'age': 'Senior',
    'dm': 1,
    'ane': 1,
    'appet': 0,
    'htn': 1
}

q = infer.query(variables=['ckd'], evidence=evidence)
print(q)

+--------+------------+
| ckd    |   phi(ckd) |
+========+============+
| ckd(0) |     0.0000 |
+--------+------------+
| ckd(1) |     1.0000 |
+--------+------------+


In [15]:
evidence2 = {
    'htn': 1, 
    'dm': 1, 
    'age': 'Young'
}
q2 = infer.query(variables = ['ane'], evidence = evidence2)
print(q2)

+----------+------------+
| ane      |   phi(ane) |
+==========+============+
| ane(0.0) |     0.7027 |
+----------+------------+
| ane(1.0) |     0.2973 |
+----------+------------+


In [16]:
evidence3 = {
    'bgr': 'Diabetic'
}
q3 = infer.query(variables = ['htn'], evidence = evidence3)
print(q3)

+----------+------------+
| htn      |   phi(htn) |
+==========+============+
| htn(0.0) |     0.6070 |
+----------+------------+
| htn(1.0) |     0.3930 |
+----------+------------+


In [17]:
evidence4 = {
    'age': 'Young',
    'appet': 0
}
q4 = infer.query(variables = ['ckd'], evidence = evidence3)
print(q4)

+--------+------------+
| ckd    |   phi(ckd) |
+========+============+
| ckd(0) |     0.3587 |
+--------+------------+
| ckd(1) |     0.6413 |
+--------+------------+
